<a href="https://colab.research.google.com/github/kimyj1503/Audio-Effects/blob/master/ProteusCapture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# GuitarML Proteus Capture Instructions
-------------
You will need the "ProteusCapture.wav" file to re-amp your gear. You can download this audio file here: [ProteusCapture.wav](https://github.com/GuitarML/Automated-GuitarAmpModelling/raw/proteus-capture/Data/Proteus_Capture.wav).

Reamp your pedal, amp, or rig by running this audio signal through your device and record the output. For best results, use a reamp box for pedals and a loadbox for amps (using a mic for amps is also an option). Render the new recording as a WAV file. **Please use 44.1kHz, 16-bit PCM, mono.**
Video tutorials are available on [Youtube](https://youtu.be/86oQuYHjpy0).

-------------

#####  **Step 1**. Click "Runtime" in the Colab top menu bar, then "Change Runtime Type" and select "GPU" and save. You have a limited number of consecutive GPU hours with the free version of Colab, but this will reset in a day or so.

#####  **Step 2**. Upload your recorded output from your device by dragging and dropping the "out.wav" file to the file browser in the left hand window in Colab. Ensure that this file is named "out.wav" for snapshot models. Parameterized capture of a knob requires 5 separate .wav output files, named:
       "out1.wav", "out2.wav", "out3.wav", "out4.wav", "out5.wav"

##### **Step 3**. Run the SETUP ENVIRONMENT section by clicking the run arrow on the top left of the box. This should take less than a minute.

##### **Step 4**. Choose one (and only one) of the next 4 options as applicable to your captured device. This will begin the model training process. It should take about 10 minutes with GPU runtime selected. The training may finish before reaching 300 epochs, this is normal. Parameterized capture of a full knob will take significantly longer, 30+ minutes.
        
##### **Step 5**. Run this section to generate a Proteus compatible model file. Download from the left hand file browser by right clicking the "newProteusModel.json" filename in the left hand menu and selecting "Download". May need to refresh the filebrowser by clicking the refresh folder icon.

##### **Step 6**. Optionally, generate a plot of the signals to visually see how close the model is to the actual device.

------------
**Note**: Recommended to "Disconnect and delete runtime" from the "Runtime" menu and "Reconnect" to reset the environment
when training additional models. Make sure to download your trained model before disconnecting.

**Note**: To continue to refine the model, you may run the same Step 4 (a,b,c) after changing the "-eps" field to the number of epochs to continue training. For example, in Step 4c., change "-eps 300" to "-eps 100" to run an additional 100 epochs. Then re-run Step 5 to generate a new Proteus compatible model.


------------
### **STEP 3** : SETUP ENVIRONMENT
###### Run this block to setup environment. May take a few minutes. Only run this once per Colab session.

In [ ]:
!git clone https://github.com/GuitarML/Automated-GuitarAmpModelling.git
%cd Automated-GuitarAmpModelling/
!git checkout proteus-capture
!git submodule update --init --recursive
!pip install torch==2.3.0 # This is a work-around for the breaking changes in pytorch 2.4.0, please allow a few extra minutes for reinstalling dependencies

Cloning into 'Automated-GuitarAmpModelling'...
remote: Enumerating objects: 644, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 644 (delta 104), reused 88 (delta 86), pack-reused 507 (from 1)
Receiving objects: 100% (644/644), 168.95 MiB | 19.83 MiB/s, done.
Resolving deltas: 100% (281/281), done.
Updating files: 100% (66/66), done.
/content/Automated-GuitarAmpModelling
Branch 'proteus-capture' set up to track remote branch 'proteus-capture' from 'origin'.
Switched to a new branch 'proteus-capture'
Submodule 'CoreAudioML' (https://github.com/GuitarML/CoreAudioML) registered for path 'CoreAudioML'
Cloning into '/content/Automated-GuitarAmpModelling/CoreAudioML'...
Submodule path 'CoreAudioML': checked out '0831965db63add6498c1849f757a95e6c3c81f33'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━

------------
### **STEP 4d**.  Run this to start capture of a Knob/Control on your amp or pedal (Parameterized Capture)
######           This requires 5 separate recordings from a Gain or EQ knob at 5 equal steps, using the Proteus_Capture.wav as input. For example: 0% (or just above 0%), 25%, 50%, 75%, 100% You must upload your five .wav files and re-name them:
               "out1.wav", "out2.wav", "out3.wav", "out4.wav", "out5.wav"

In [ ]:
model="proteus_knob"
!python prep_wav2.py $model -p "./Configs/Parameterization-Config-Proteus.json" --normalize true
!python dist_model_recnet.py -l "RNN3-"$model -is 2 -eps 300

------------
### **STEP 5**. After Above Capture Process is Completed, Run this to generate a Proteus compatible model file.
######The model file will be located in the top level folder with the name "newProteusModel.json" May need to refresh the left hand file browser by clicking the refresh folder icon. Right click the file and choose download, then rename as appropriate.


In [ ]:
%cp "Results/"$model"-RNN3-"$model"/model_best.json" ../newProteusModel.json

------------
### **STEP 6**. (Optional) After above Capture Process is completed, Run this to generate graphs of the device model
######         The PNG image will be located in the top level folder with the name "detail_signal_comparison_e2s...". May need to refresh the left hand file browser by clicking the refresh folder icon. Double click the PNG file to view the graphs. These graphs show approximately 10 milliseconds of audio signal. The top plot shows the input signal. The middle plot shows the target device signal vs the signal predicted by the new model. The goal is for these signals to be as close as possible.

In [ ]:
!python plot.py $model
%cp "Results/"$model"-RNN3-"$model"/"$model"_Detail"* ../

------------
### TROUBLESHOOTING NOTES

1. Training will go through 300 epochs, beginning with a pre-trained starting point model. The number of epochs can be adjusted by changing the "-eps" value in the dist_model_recnet.py line on step 4 (a,b,c,d). You can run the same training step again to refine the model, and stop at any time.

2. This capture method is limited to non-time based effects (no Delay, Reverb, Flange, Phaser, etc.) It is intended for Amplifiers, Distortion, Overdrive, Boost, Preamps. Results on Compressors vary. The same rules apply for capturing a plugin or other digital audio effect.

3. For ideal amplifier capture, record the direct out signal from a load box. Recommended to use a re-amp device when playing the ProteusCapture.wav out of your audio device for impedence matching.

4. The final loss values during training should be less than 0.08 to be considered successful. A loss of 0.02 or less is ideal. Higher than 0.10 indicates difficulty in training a model from the device, but note that mic'd amps will typically have a higher loss due to the complexities of the cab/mic. If the loss remains at 0.75 or higher, this could indicate a problem with the out.wav file, either from audio misalignment or error during the upload to Colab. If the latency in your recording is too high (more than 3 milliseconds) use the click around 1 second into the capture WAV to line up your out.wav in your DAW.

5. For Parameterized Capture of an amp or pedal knob, if the device knob at 0% means there is no volume out of your device (such as a gain knob on an amplifier), then you should raise it slightly until your hear volume out. The model training doesn't do well if one of the 5 steps is silent.

6. Email "smartguitarml@gmail.com" for troubleshooting help, or to share Proteus models to add to the ToneLibrary.